# Train, test, export model for Electricty monitor IDS 

Datasets is from personal usage documentation
<br> <br>
Clustered with unsupervised algorithm - Isolation Foreset
<br>
Feature classifying/learning with supervised algorithm - Random Forest

## Config

### Import

In [2]:
# Import necessary libraries
import pandas as pd
import numpy as np
import itertools
import matplotlib.pyplot as plt
import datetime
import joblib

import mysql.connector
from sklearn.preprocessing import StandardScaler
from sklearn.cluster import DBSCAN
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import classification_report, confusion_matrix, recall_score, roc_auc_score, make_scorer, accuracy_score, precision_score, f1_score
from sklearn.utils import resample
from sklearn.ensemble import IsolationForest, RandomForestClassifier
from sklearn.neighbors import LocalOutlierFactor
from sklearn.model_selection import GridSearchCV, train_test_split
from sklearn.svm import OneClassSVM
from sklearn.neighbors import NearestNeighbors
import matplotlib.pyplot as plt
import seaborn as sns

### database config

In [3]:
# Configure database connection
db_config = {
    'host': 'localhost',
    'user': 'root',
    'password': '',
    'database': 'db_esp32'
}

In [4]:
# Fetch data from the database with only power > 0 and order by created_at in descending order
def fetch_data():
    conn = mysql.connector.connect(**db_config)
    query = f"SELECT * FROM electricity_monitor WHERE power > 0.5 ORDER BY created_at DESC"
    df = pd.read_sql(query, conn)
    conn.close()
    return df

def get_deviceId():
    conn = mysql.connector.connect(**db_config)
    query = f"SELECT DISTINCT device_id FROM electricity_monitor"
    device_ids = pd.read_sql(query, conn)
    return device_ids

df = fetch_data()
device_ids = get_deviceId()

print(df.count())

C:\Users\Sabian\AppData\Local\Temp\ipykernel_19880\1538424201.py:5: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql(query, conn)


data_id         14342
device_id       14342
voltage         14342
current         14342
power           14342
energy          14342
frequency       14342
power_factor    14342
created_at      14342
dtype: int64


C:\Users\Sabian\AppData\Local\Temp\ipykernel_19880\1538424201.py:12: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  device_ids = pd.read_sql(query, conn)


## Pre-processing

### Load the datasets

In [5]:
print('Count: \n', df.count())
print('\ndatasets: \n', df.head())
print('\n')
print('device_ids: \n', device_ids)

Count: 
 data_id         14342
device_id       14342
voltage         14342
current         14342
power           14342
energy          14342
frequency       14342
power_factor    14342
created_at      14342
dtype: int64

datasets: 
    data_id          device_id  voltage  current  power  energy  frequency  \
0   122335  D4:8A:FC:60:47:60    196.1    0.027    2.0   0.832       49.9   
1   122334  D4:8A:FC:60:47:60    197.8    0.026    2.0   0.832       49.9   
2   122333  D4:8A:FC:60:47:60    197.9    0.027    2.1   0.832       49.9   
3   122332  D4:8A:FC:60:47:60    197.0    0.027    2.1   0.832       49.9   
4   122331  D4:8A:FC:60:47:60    197.0    0.026    2.0   0.832       49.9   

   power_factor          created_at  
0          0.38 2024-07-10 09:36:18  
1          0.39 2024-07-10 09:36:13  
2          0.39 2024-07-10 09:36:08  
3          0.39 2024-07-10 09:36:03  
4          0.39 2024-07-10 09:35:58  


device_ids: 
            device_id
0  D4:8A:FC:60:47:60


### Convert datetime into numeric

In [6]:
# df['created_at'] = df['created_at'].astype('int64') // 10**9


#### Handling missing values

In [7]:
print("before:\n\n", df.isnull().sum())
# Fill missing values with median
df['voltage'].fillna(df['voltage'].median())

# Fill device_id with the latest id
latest_id = device_ids['device_id'].iloc[-1]
df['device_id'].fillna(latest_id)

# Fill created_at with current date and time
df['created_at'].fillna(pd.to_datetime('now'))

print("\n\nafter:\n", df.isnull().sum())

before:

 data_id         0
device_id       0
voltage         0
current         0
power           0
energy          0
frequency       0
power_factor    0
created_at      0
dtype: int64


after:
 data_id         0
device_id       0
voltage         0
current         0
power           0
energy          0
frequency       0
power_factor    0
created_at      0
dtype: int64


## Model Training

### Split dataset for training and testing

In [8]:
train_df, test_df = train_test_split(df, test_size=0.33, random_state=42)

print(train_df.count())

data_id         9609
device_id       9609
voltage         9609
current         9609
power           9609
energy          9609
frequency       9609
power_factor    9609
created_at      9609
dtype: int64


In [9]:
# Inspect the distribution of the power feature
print("Power Distribution in Training Data:")
print(train_df['power'].describe())

Power Distribution in Training Data:
count    9.609000e+03
mean     2.175999e+04
std      2.131282e+06
min      6.000000e-01
25%      1.090000e+01
50%      1.680000e+01
75%      2.020000e+01
max      2.089200e+08
Name: power, dtype: float64


### Unsupervised Learning - Isolation Forest

### Train and predict 

In [18]:
def train_if():
    for device_id in device_ids['device_id']:
        features = ['voltage', 'current', 'power', 'energy', 'frequency', 'power_factor']
        # Train model
        if_model = IsolationForest(contamination=0.1, random_state=42)
        if_model.fit(train_df[train_df['device_id'] == device_id][features])

        # Predict Label
        df.loc[df['device_id'] == device_id, 'if_label'] = if_model.predict(df[df['device_id'] == device_id][features])
        # Explicitly cast the 'if_label' column to string type
        df['if_label'] = df['if_label'].astype(str)
        # Conditional assignment using np.where
        df.loc[df['device_id'] == device_id, 'if_label'] = np.where(df['if_label'] == '1.0', 'normal', 'anomaly')

        score = if_model.decision_function(df[df['device_id'] == device_id][features])
        print(f"Score: {score}")
        filename = f'if_model_{device_id.replace(":", "-")}.pkl'
        joblib.dump(if_model, filename)
        
         # Predict anomaly scores
        scores = if_model.decision_function(df[df['device_id'] == device_id][features])
    
train_if()

Score: [-0.07354423 -0.06092005 -0.05639264 ...  0.06359868  0.111713
  0.11469054]


### Show results for rows that have normal label

In [11]:
print(df[df['if_label'] == 'normal'].head(10))


    data_id          device_id  voltage  current  power  energy  frequency  \
12   122323  D4:8A:FC:60:47:60    199.8    0.030    2.5   0.832       50.0   
13   122322  D4:8A:FC:60:47:60    198.3    0.033    2.9   0.832       50.0   
14   122321  D4:8A:FC:60:47:60    200.2    0.031    2.6   0.832       50.0   
15   122320  D4:8A:FC:60:47:60    199.4    0.030    2.5   0.832       50.0   
17   122318  D4:8A:FC:60:47:60    200.1    0.027    2.1   0.832       50.0   
26   121601  D4:8A:FC:60:47:60    206.2    0.345   42.4   0.815       50.0   
32   121505  D4:8A:FC:60:47:60    206.1    0.379   47.8   0.813       50.0   
33   121504  D4:8A:FC:60:47:60    205.5    0.381   47.9   0.813       50.0   
34   121503  D4:8A:FC:60:47:60    205.9    0.382   48.1   0.813       50.0   
35   121502  D4:8A:FC:60:47:60    205.6    0.383   48.3   0.813       50.0   

    power_factor          created_at if_label  
12          0.42 2024-07-10 09:35:18   normal  
13          0.44 2024-07-10 09:35:13   normal

In [12]:
# testing apakah power di atas 20 watt ada yang normal
print(df[(df['if_label'] != 'normal') & (df['power'] > 50) & (df['power'] < 60)].head(10))


    data_id          device_id  voltage  current  power  energy  frequency  \
21   121606  D4:8A:FC:60:47:60    207.9    0.415   53.5   0.816       50.0   
22   121605  D4:8A:FC:60:47:60    208.4    0.416   53.7   0.816       50.0   
23   121604  D4:8A:FC:60:47:60    208.2    0.418   53.9   0.816       50.0   
24   121603  D4:8A:FC:60:47:60    205.9    0.421   53.9   0.815       50.0   
25   121602  D4:8A:FC:60:47:60    205.4    0.421   53.7   0.815       50.0   
46   121491  D4:8A:FC:60:47:60    204.5    0.397   50.1   0.813       49.9   
47   121490  D4:8A:FC:60:47:60    204.4    0.400   50.3   0.812       49.9   
49   121488  D4:8A:FC:60:47:60    203.2    0.412   50.7   0.812       49.9   
50   121487  D4:8A:FC:60:47:60    203.0    0.414   50.9   0.812       49.9   
52   121485  D4:8A:FC:60:47:60    203.7    0.408   51.3   0.812       49.9   

    power_factor          created_at if_label  
21          0.62 2024-07-09 23:55:43  anomaly  
22          0.62 2024-07-09 23:55:38  anomaly

### Show results for rows that have anomaly label

In [13]:
print(df[df['if_label'] == 'anomaly'].head(10))

   data_id          device_id  voltage  current  power  energy  frequency  \
0   122335  D4:8A:FC:60:47:60    196.1    0.027    2.0   0.832       49.9   
1   122334  D4:8A:FC:60:47:60    197.8    0.026    2.0   0.832       49.9   
2   122333  D4:8A:FC:60:47:60    197.9    0.027    2.1   0.832       49.9   
3   122332  D4:8A:FC:60:47:60    197.0    0.027    2.1   0.832       49.9   
4   122331  D4:8A:FC:60:47:60    197.0    0.026    2.0   0.832       49.9   
5   122330  D4:8A:FC:60:47:60    196.4    0.026    1.9   0.832       49.9   
6   122329  D4:8A:FC:60:47:60    195.1    0.027    2.1   0.832       49.9   
7   122328  D4:8A:FC:60:47:60    195.7    0.027    2.1   0.832       49.9   
8   122327  D4:8A:FC:60:47:60    197.2    0.026    2.0   0.832       49.9   
9   122326  D4:8A:FC:60:47:60    196.4    0.027    2.1   0.832       50.0   

   power_factor          created_at if_label  
0          0.38 2024-07-10 09:36:18  anomaly  
1          0.39 2024-07-10 09:36:13  anomaly  
2          

In [14]:
# testing apakah power di atas 17 di bawah 20 watt ada yang anomaly
print(df[(df['if_label'] != 1.0) & (df['power'] > 17) & (df['power'] < 20)].head(10))


     data_id          device_id  voltage  current  power  energy  frequency  \
225   118317  D4:8A:FC:60:47:60    205.8    0.145   17.3   0.802       50.0   
226   118316  D4:8A:FC:60:47:60    204.3    0.146   17.3   0.802       49.9   
227   118315  D4:8A:FC:60:47:60    204.2    0.148   17.5   0.802       50.0   
228   118314  D4:8A:FC:60:47:60    204.8    0.146   17.4   0.802       49.9   
229   118313  D4:8A:FC:60:47:60    204.1    0.145   17.3   0.802       49.9   
230   118312  D4:8A:FC:60:47:60    204.8    0.147   17.6   0.802       49.9   
231   118311  D4:8A:FC:60:47:60    204.9    0.146   17.4   0.802       49.9   
232   118310  D4:8A:FC:60:47:60    204.0    0.145   17.3   0.802       49.9   
233   118309  D4:8A:FC:60:47:60    204.3    0.146   17.3   0.802       49.9   
234   118308  D4:8A:FC:60:47:60    203.5    0.145   17.1   0.802       49.9   

     power_factor          created_at if_label  
225          0.58 2024-07-09 17:17:07   normal  
226          0.58 2024-07-09 17:

## Supervised Learning - Random Forest

In [16]:
train_df, test_df = train_test_split(df, test_size=0.33, random_state=42)
print(train_df.head())

       data_id          device_id  voltage  current  power  energy  frequency  \
13652    20625  D4:8A:FC:60:47:60    200.1    0.173   20.4   1.114       50.0   
12480    36966  D4:8A:FC:60:47:60    202.7    0.132   16.5   1.347       50.0   
5151     86048  D4:8A:FC:60:47:60    209.3    0.136   16.6   0.663       50.0   
2804     95136  D4:8A:FC:60:47:60    218.3    0.029    2.4   0.745       50.0   
2211    108982  D4:8A:FC:60:47:60    211.3    0.161   20.3   0.756       50.0   

       power_factor          created_at if_label  
13652          0.59 2024-05-31 22:40:09   normal  
12480          0.62 2024-06-08 13:52:48   normal  
5151           0.58 2024-07-03 19:23:36   normal  
2804           0.38 2024-07-06 20:19:01  anomaly  
2211           0.60 2024-07-07 23:54:20   normal  


In [17]:
def train_rf():
    for device_id in device_ids['device_id']:
        features = ['voltage', 'current', 'power', 'energy', 'frequency', 'power_factor']
        X = train_df[train_df['device_id'] == device_id][features]
        y = train_df[train_df['device_id'] == device_id]['if_label']
        
        # Ensure there is data to train on
        if not X.empty and not y.empty:
            rf_model = RandomForestClassifier(n_estimators=100, random_state=42)
            rf_model.fit(X, y)

            filename = f'rf_model_{device_id.replace(":", "-")}.pkl'
            joblib.dump(rf_model, filename)
            
            # Getting and printing feature importances
            feature_importances = rf_model.feature_importances_
            print(f"Feature importances for device {device_id}:")
            for feature, importance in zip(features, feature_importances):
                print(f"{feature}: {importance:.4f}")
        else:
            print(f"No data available for device {device_id}, skipping...")

train_rf()

Feature importances for device D4:8A:FC:60:47:60:
voltage: 0.0620
current: 0.2182
power: 0.2573
energy: 0.0497
frequency: 0.0735
power_factor: 0.3392
